# LeetCode 1768 — Merge Strings Alternately

This notebook collects multiple solutions (from baseline to more optimized) for the same problem and includes quick tests and micro-benchmarks.

**Problem (LeetCode 1768):**
Given two strings `word1` and `word2`, merge them by adding letters in alternating order, starting with `word1`. If a string is longer than the other, append the additional letters onto the end of the merged string.

Return the merged string.

# Stats

![Merge Example](./images/leetcode_1768_merge_strings_alternately.png)

## Solutions Overview
We'll document and compare the following Python solutions:

1. **Baseline (reverse + pop)** — builds reversed lists and pops from the end.
2. **Concat-in-loop (slow)** — concatenates directly into a string inside the loop (documented as an anti-pattern for performance).
3. **Two-pointer + list builder** — index both strings, append to a list, and `''.join()` once at the end.
4. **`itertools.zip_longest` + `extend`** — succinct and stays in C for most operations.
5. **`zip` + `chain.from_iterable`** — very few Python-level ops for the interleaved head, then append remainder slices.

All are **O(n + m)**; the main differences are constant factors and memory overhead.

In [1]:
from itertools import zip_longest, chain

def solution_baseline_reverse_pop(word1: str, word2: str) -> str:
    """Reverse both strings into lists and pop from the end.
    Mirrors the initial approach while avoiding extra class wrapper for clarity here.
    """
    combined = []
    w1 = list(reversed(list(word1)))
    w2 = list(reversed(list(word2)))
    while w1 or w2:
        if w1:
            combined.append(w1.pop())
        if w2:
            combined.append(w2.pop())
    return ''.join(combined)

def solution_concat_in_loop(word1: str, word2: str) -> str:
    """Concat into a Python string in the loop (intentionally slower)."""
    n1, n2 = len(word1), len(word2)
    mx = n1 if n1 > n2 else n2
    out = ''
    for i in range(mx):
        if i < n1:
            out += word1[i]
        if i < n2:
            out += word2[i]
    return out

def solution_two_pointer_list(word1: str, word2: str) -> str:
    """Two pointers + list appends; append the remainder as a slice in one step."""
    n1, n2 = len(word1), len(word2)
    i = 0
    out = []
    append = out.append
    mn = n1 if n1 < n2 else n2
    while i < mn:
        append(word1[i])
        append(word2[i])
        i += 1
    if i < n1:
        append(word1[i:])
    elif i < n2:
        append(word2[i:])
    return ''.join(out)

def solution_zip_longest_extend(word1: str, word2: str) -> str:
    """Use zip_longest over both strings; extend list per pair; join once."""
    out = []
    extend = out.extend
    for a, b in zip_longest(word1, word2, fillvalue=''):
        if a: extend(a)
        if b: extend(b)
    return ''.join(out)

def solution_zip_chain(word1: str, word2: str) -> str:
    """Interleave head with zip + chain (stays in C), then add remainders."""
    mn = min(len(word1), len(word2))
    head = ''.join(chain.from_iterable(zip(word1[:mn], word2[:mn])))
    return head + word1[mn:] + word2[mn:]

## Quick Correctness Checks
We run a series of asserts comparing all solutions on a set of edge cases and random cases.

In [2]:
import random
import string

def merge_reference(word1: str, word2: str) -> str:
    # A straightforward reference implementation using indexing
    n1, n2 = len(word1), len(word2)
    i = 0
    out = []
    mn = min(n1, n2)
    while i < mn:
        out.append(word1[i])
        out.append(word2[i])
        i += 1
    if i < n1:
        out.append(word1[i:])
    elif i < n2:
        out.append(word2[i:])
    return ''.join(out)

tests = [
    ("", ""),
    ("a", ""),
    ("", "b"),
    ("ab", "cd"),
    ("abc", "pq"),
    ("ab", "pqr"),
    ("abc", ""),
    ("", "xyz"),
]

# Add random tests
rng = random.Random(42)
alphabet = string.ascii_lowercase
for _ in range(50):
    n1 = rng.randint(0, 50)
    n2 = rng.randint(0, 50)
    w1 = ''.join(rng.choice(alphabet) for _ in range(n1))
    w2 = ''.join(rng.choice(alphabet) for _ in range(n2))
    tests.append((w1, w2))

funcs = [
    solution_baseline_reverse_pop,
    solution_concat_in_loop,
    solution_two_pointer_list,
    solution_zip_longest_extend,
    solution_zip_chain,
]

for w1, w2 in tests:
    ref = merge_reference(w1, w2)
    for f in funcs:
        out = f(w1, w2)
        assert out == ref, f"Mismatch in {f.__name__}: {w1!r}, {w2!r}, got {out!r}, expected {ref!r}"

print("All solutions match the reference on edge + random tests.")

All solutions match the reference on edge + random tests.


## Micro-benchmarks
We benchmark each function on random inputs across a range of lengths. Numbers are illustrative for your machine; relative ordering is what matters.

In [3]:
import timeit
import statistics as stats
import pandas as pd
import random, string

def make_case(n1: int, n2: int, seed=0):
    rng = random.Random(seed)
    s = string.ascii_letters
    return (
        ''.join(rng.choice(s) for _ in range(n1)),
        ''.join(rng.choice(s) for _ in range(n2)),
    )

cases = [
    (10, 10),
    (100, 100),
    (1000, 1000),
    (2000, 1000),
    (1000, 2000),
]

functions = {
    'baseline_reverse_pop': solution_baseline_reverse_pop,
    'concat_in_loop (slow)': solution_concat_in_loop,
    'two_pointer_list': solution_two_pointer_list,
    'zip_longest_extend': solution_zip_longest_extend,
    'zip_chain': solution_zip_chain,
}

rows = []
for n1, n2 in cases:
    w1, w2 = make_case(n1, n2, seed=n1*10+n2)
    for name, fn in functions.items():
        # time each function a few times and take median
        t = timeit.repeat(lambda: fn(w1, w2), repeat=7, number=1000)
        rows.append({
            'case': f'{n1}x{n2}',
            'function': name,
            'median_ms_per_1k': 1000 * stats.median(t),
            'min_ms_per_1k': 1000 * min(t),
            'max_ms_per_1k': 1000 * max(t),
        })

df = pd.DataFrame(rows).sort_values(['case', 'median_ms_per_1k'])
df

,case,function,median_ms_per_1k,min_ms_per_1k,max_ms_per_1k
14,1000x1000,zip_chain,70.974070,64.783142,85.806862
10,1000x1000,baseline_reverse_pop,90.328109,87.000988,117.007970
12,1000x1000,two_pointer_list,101.505289,91.564713,148.148636
13,1000x1000,zip_longest_extend,156.549238,145.857285,188.641199
11,1000x1000,concat_in_loop (slow),156.657534,120.523132,189.448704
24,1000x2000,zip_chain,65.243480,64.724994,66.258973
22,1000x2000,two_pointer_list,91.616690,90.634981,93.757019
20,1000x2000,baseline_reverse_pop,157.530781,153.406274,158.618407
21,1000x2000,concat_in_loop (slow),200.269029,194.973523,225.934811
23,1000x2000,zip_longest_extend,257.147169,224.668752,271.039687


### Takeaways (typical)
- Avoid string concatenation inside loops — it tends to be the slowest.
- Approaches that keep most work in C (`zip`/`chain`/`join`) generally perform best.
- The two-pointer list-builder is also consistently strong and very readable.

Your machine may differ a bit, but relative order usually holds.

## Production-Style Class Implementations (drop-in for LeetCode)
These mirror the faster approaches in `class Solution:` form if you want to paste into LeetCode.

In [4]:
class SolutionTwoPointer:
    def mergeAlternately(self, word1: str, word2: str) -> str:
        n1, n2 = len(word1), len(word2)
        i = 0
        out = []
        append = out.append
        mn = n1 if n1 < n2 else n2
        while i < mn:
            append(word1[i])
            append(word2[i])
            i += 1
        if i < n1:
            append(word1[i:])
        elif i < n2:
            append(word2[i:])
        return ''.join(out)

class SolutionZipChain:
    def mergeAlternately(self, word1: str, word2: str) -> str:
        mn = min(len(word1), len(word2))
        head = ''.join(chain.from_iterable(zip(word1[:mn], word2[:mn])))
        return head + word1[mn:] + word2[mn:]

## Environment Info

In [5]:
import sys, platform
print({'python': sys.version, 'platform': platform.platform(), 'timestamp': '2025-09-20T22:30:18.510841Z'})

{'python': '3.12.1 (main, Jul 10 2025, 11:57:50) [GCC 13.3.0]', 'platform': 'Linux-6.8.0-1030-azure-x86_64-with-glibc2.39', 'timestamp': '2025-09-20T22:30:18.510841Z'}


# Appendix: Methods & Functions Explained

This section documents the Python list methods and standard-library helpers referenced in the solutions, with succinct explanations, time complexity notes, and tiny examples.

## Core string/list ops

### `''.join(iterable)`
- **What**: Builds a string by concatenating all pieces from an iterable of strings.
- **Why**: More efficient than `+=` in a loop because it allocates once.
- **Complexity**: O(total length of the result).
- **Example**:
```python
pieces = ['a', 'b', 'c']
''.join(pieces)  # 'abc'
```

### Slicing (`s[i:]`, `s[:mn]`)
- **What**: Returns a substring (or sublist) by range.
- **Why**: Appending the remainder as a *single slice* reduces Python-level loop overhead.
- **Complexity**: O(length of slice) to copy.
- **Example**:
```python
s = 'hello'
s[2:]    # 'llo'
s[:3]    # 'hel'
```

### `reversed(x)` and `list(reversed(x))`
- **What**: `reversed` gives a reverse iterator over a *sequence*; wrapping with `list(...)` materializes it.
- **Why**: Used in the baseline to pop from the end; creates an extra list copy.
- **Complexity**: Materializing: O(n) time & memory.
- **Example**:
```python
list(reversed('abc'))  # ['c', 'b', 'a']
```

### `list.pop()`
- **What**: Removes and returns an element. `pop()` defaults to the last element.
- **Why**: Efficient for stack-style access from the end of a list.
- **Complexity**: `pop()` from end is **amortized O(1)**; from front is O(n).
- **Example**:
```python
a = ['x', 'y', 'z']
a.pop()   # 'z'; a -> ['x', 'y']
```

### `list.append(x)`
- **What**: Adds a single item to the end of a list.
- **Why**: Primary way we build up a result before one final `''.join(...)`.
- **Complexity**: Amortized **O(1)**.
- **Example**:
```python
out = []
out.append('a'); out.append('b')  # out -> ['a', 'b']
```

### `list.extend(iterable)`
- **What**: Adds all items from an iterable to the end of a list.
- **Why**: Useful when appending multiple items (e.g., a short string slice) in one call.
- **Complexity**: O(k) where k is the number of extended elements.
- **Example**:
```python
out = ['a']
out.extend('bc')  # out -> ['a', 'b', 'c']
```

## `itertools` helpers

### `zip(a, b)`
- **What**: Pairs elements from iterables until the shortest is exhausted.
- **Why**: Interleaving the *common length* prefix efficiently in C.
- **Complexity**: O(min(len(a), len(b))).
- **Example**:
```python
list(zip('ab', 'XY'))  # [('a','X'), ('b','Y')]
```

### `itertools.zip_longest(a, b, fillvalue='')`
- **What**: Like `zip`, but continues to the longest input, using `fillvalue` when one iterable is exhausted.
- **Why**: Lets us iterate pairs to the very end without manual bounds checks.
- **Complexity**: O(max(len(a), len(b))).
- **Example**:
```python
from itertools import zip_longest
list(zip_longest('ab', 'WXYZ', fillvalue=''))
# [('a','W'), ('b','X'), ('','Y'), ('','Z')]
```

### `itertools.chain.from_iterable(iterable_of_iterables)`
- **What**: Flattens one level of nesting from an iterable of iterables.
- **Why**: Turning pairs from `zip(...)` into a single stream of characters before a single `''.join(...)` keeps work in C.
- **Complexity**: O(total number of elements produced).
- **Example**:
```python
from itertools import chain
list(chain.from_iterable([('a','X'), ('b','Y')]))  # ['a', 'X', 'b', 'Y']
```

## Performance notes & gotchas
- Avoid `out += char` repeatedly inside loops — it creates many intermediate strings.
- Prefer building a list of pieces, then `''.join(...)` once.
- Use `zip` when you only need to go to the **shorter** length, and `zip_longest` when you must cover the **longer**.
- Appending the tail as a **single slice** (`out.append(word[i:])`) keeps Python-level calls low.


---
**Notes:**
- All implementations are `O(n + m)`; differences are constant factors.
- Prefer building pieces into a list and `''.join()` once.
- Where possible, leverage built-ins (`zip`, `itertools`, slicing) to keep work in C.